In [3]:
import gensim
from gensim import corpora, models, similarities
import nltk
import json

In [4]:
from collections import OrderedDict
from semanticparser import *
from tools import *

In [5]:
username = 'franklin'

In [27]:
len(texts)

1984

In [17]:
with open('data/%s.txt' % username) as docs_file:
    documents = docs_file.read().replace('\n', '').decode('utf-8')

# break into sentences
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sents_all = sent_detector.tokenize(documents.strip())
sent_generator = nltk.bigrams(sents_all)
sents = [" ".join(s) for s in sent_generator]


In [23]:
texts = remove_stopwords(sents, True)
dictionary, lsi, index = texts_to_index(texts, 8, username)

In [18]:
with open('data/%s.json' % username, 'w') as f:
    json.dump(sents,f)

In [24]:
dictionary.save('data/%s.dict' % username) 
lsi.save('data/%s-corpus.lsi' % username)
index.save('data/%s-corpus.index' % username)

In [31]:
### test functions
def load_sents(username, root='data'):
    """ Load documents
            Preprocessed: dictionary, corpus, index, lsi
            Archives: documents
    """
    dictionary = corpora.Dictionary.load('%s/%s.dict' % (root,username))

    with open('%s/%s.json' % (root,username)) as docs_file:
        documents = json.load(docs_file)

    lsi = models.LsiModel.load('%s/%s-corpus.lsi' % (root,username))
    #index = similarities.MatrixSimilarity.load('data/%s-corpus.index' % username)
    index = similarities.Similarity.load('%s/%s-corpus.index' % (root,username))

    return documents, dictionary, lsi, index

def test_response(username, t):
    trash = [t]
    documents0, dictionary0, lsi0, index0 = load_sents(username)
    r = gen_response(documents0, dictionary0, lsi0, index0, t, trash, True)
    print r

def gen_response(sents, dictionary, lsi, index, t, trash, limit1=True):
    # tokenize input sentence
    clean_input = clean_str(t).lower().split()
    
    print clean_input

    # get most similar post from input sentence
    sims = query_page(clean_input, dictionary, lsi, index)

    # repeat the process on the sentences in the doc
    sample = [sents[sims[0][0]], sents[sims[1][0]], sents[sims[2][0]], sents[sims[3][0]],
                sents[sims[4][0]], sents[sims[5][0]], sents[sims[6][0]], sents[sims[7][0]],
                 sents[sims[8][0]],  sents[sims[9][0]],  sents[sims[10][0]],  sents[sims[11][0]],
                  sents[sims[12][0]], sents[sims[13][0]],  sents[sims[14][0]],  sents[sims[15][0]]]

    rmult = []
    #reply = create_reply(sample, '')
    if sample and sample not in trash:
        # reply to the tweet
        if limit1:
            return sample
        else:
            rmult.append(sample)
    if limit1:
        return None
    else:
        return rmult


In [35]:
test_response('franklin', "do you have any good drugs?")

['do', 'you', 'have', 'any', 'good', 'drugs?']
[u'"But that the most acceptable service of God is doing good to man. "That the soul is immortal.', u'good green tea. butter.', u'"That he ought to be worshiped by adoration, prayer, and thanksgiving. "But that the most acceptable service of God is doing good to man.', u'FRUGALITY. Make no expense but to do good to others or yourself;\ri.e., waste nothing.', u'good\r  1 lb. good green tea.', u"Paying a due respect to God's ministers. These\rmight be all good things; but, as they were not the kind of good things\rthat I expected from that text, I despaired of ever meeting with them\rfrom any other, was disgusted, and attended his preaching no more.", u'To which I have besides some other inducements. Having emerged from the poverty and obscurity in which I was born and\rbred, to a state of affluence and some degree of reputation in the\rworld, and having gone so far through life with a considerable share of\rfelicity, the conducing means I m